# senti preprocess 3

In [1]:
import os
import time
import pickle
import pandas as pd
import numpy as np

datasets_name = 'Cell_Phones_and_Accessories'
data_path = 'dict/' + datasets_name +'/session_data.csv'

price_num = '100'

data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID', 'priceLevel', 'category']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_inter_num = item_inter_num.reset_index()[['sessionID', 'itemID']]
# item_num=item_inter_num.rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>4]
# data = data[['sessionID', 'itemID', 'priceLevel']]

# dict (sessionID:[itemID,itemID])
sess_all = {}
# dict (sessionID:[priceLevel, priceLevel])
price_all = {}



# dict (sessionID:[cate, cate])
cate_all = {}
# dict (sessionID:[brand, brand])
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    price = row['priceLevel']
    cate = row['category']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        price_all[sess_id].append(price)
        cate_all[sess_id].append(cate)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        price_all[sess_id] = []
        price_all[sess_id].append(price)
        cate_all[sess_id] = []
        cate_all[sess_id].append(cate)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() 
tra_price = dict()# dict(session_id:[price,price])
tes_price = dict()
tra_cate = dict()# dict(session_id:[cate,cate])
tes_cate = dict()
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    all_price = price_all[sess_temp]
    all_cate = cate_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
        all_price = all_price[:20]
        all_cate = all_cate[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
        tra_price[sess_temp] = all_price
        tra_cate[sess_temp] = all_cate
    else:
        tes_sess[sess_temp] = all_seqs
        tes_price[sess_temp] = all_price
        tes_cate[sess_temp] = all_cate

# the number of item & price, renumber
# item_ctr = 1
# price_ctr = 1
# cate_ctr = 1

item_dict = {} #dict(old_itemID: new_itemID)
cate_dict = {} #dict(old_cate: new_cate)
price_dict = {} #dict(old_price: new_price)


# item_id: price_level, item_id: rating
item_price = {} #dict[new_itemID: priceLevel]
item_cate = {} #dict[new_itemID: cate]




# tra_sess tra_price tra_cate
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
#     global item_ctr
#     print(item_ctr)
    train_seqs = []
    train_price = []
    train_cate = []
#     train_brand = []
    item_ctr = 1
    price_ctr = 1
    cate_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        price_seq = tra_price[s]
        cate_seq = tra_cate[s]
        outseq = []
        pri_outseq = []
#         rating_outseq = []
        cate_outseq = []
        for (i, j_pri, w_cate) in zip(seq, price_seq, cate_seq):
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
            if j_pri in price_dict:
                pri_outseq += [price_dict[j_pri]]
            else:
                pri_outseq += [price_ctr]
                price_dict[j_pri] = price_ctr
                price_ctr += 1
            if w_cate in cate_dict:
                cate_outseq += [cate_dict[w_cate]]
            else:
                cate_outseq += [cate_ctr]
                cate_dict[w_cate] = cate_ctr
                cate_ctr += 1
            if item_dict[i] not in item_price:
                item_price[item_dict[i]] = price_dict[j_pri]
                item_cate[item_dict[i]] = cate_dict[w_cate]
        if len(outseq) < 2:  # Doesn't occur
            print('session length is 1')
            continue
        train_seqs += [outseq]
        train_price += [pri_outseq]
        train_cate += [cate_outseq]
    print("#train_session",len(train_seqs))
    print("#train_items",item_ctr-1)
    print("#train_price",price_ctr-1)
    print("#train_category",cate_ctr-1)
    return train_seqs, train_price, train_cate


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    test_price = []
    test_cate = []
    for s in tes_sess:
        outseq = []
        out_price = []
        out_rating = []
        out_cate = []
        for i, j, w in zip(tes_sess[s], tes_price[s], tes_cate[s]):
            if i in item_dict:
                outseq += [item_dict[i]]
                out_price += [price_dict[j]]
                out_cate += [cate_dict[w]]
        if len(outseq) < 2:
            print('obtain test session length is 1')
            continue
        test_seqs += [outseq]
        test_price += [out_price]
        test_cate += [out_cate]
    print("#test_session: ",len(test_seqs))
    return test_seqs, test_price, test_cate

# Convert test sessions to sequences, renumbering items that do not appear in training set
def obtian_tes_cold():
    test_seqs = []
    test_price = []
    test_seqs_cold = []
    test_price_cold = []
    num_cold = 0
    global item_ctr,price_ctr
#     test_cate = []
    for s in tes_sess:
        outseq = []
        out_price = []
        contain_cold = False
        for i, p in zip(tes_sess[s], tes_price[s]):
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                num_cold += 1
                contain_cold = True
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
            if p in price_dict:
                out_price += [price_dict[p]]
            else:
                out_price += [price_ctr]
                price_dict[p]= price_ctr
                price_ctr += 1
#             if i in item_dict:
#                 outseq += [item_dict[i]]
#                 out_price += [price_dict[j]]

        if len(outseq) < 2:
            print('obtain test session length is 1')
            continue
        if contain_cold:
            test_seqs_cold += [outseq]
            test_price_cold += [out_price]
        else:
            test_seqs += [outseq]
            test_price += [out_price]
    print("#test_session: ",len(test_seqs))
    print("#cold_test_session: ",len(test_seqs_cold))
    print('#cold_items',str(num_cold))
    return test_seqs, test_price, test_seqs_cold, test_price_cold


# 数据增强
def process_seqs(iseqs, iprice, icate):
    out_seqs = []
    out_price = []
    out_cate = []
    labs = []
    max_length = 19
    for seq, pri, rat, cat in zip(iseqs, iprice, icate):
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
            out_price += [pri[:-i]]
            out_cate += [cat[:-i]]
    return out_seqs, out_price, out_cate, labs

# 无数据增强
def process_seqs_no(iseqs, iprice, icate):
    print("no data augment")
    out_seqs = []
    out_price = []
    out_cate = []
    labs = []
    max_length = 19
    for seq, pri, cat in zip(iseqs, iprice, icate):     
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
        out_price += [pri[:-1]]
        out_cate += [cat[:-1]]
    return out_seqs, out_price, out_cate, labs

tra_seqs, tra_pri, tra_cat  = obtian_tra()
tes_seqs, tes_pri, tes_cat  = obtian_tes()
# save item_dict:(old_itemID: new_itemID) nocold
old2new_nocold_path = './dict/' + datasets_name + '/old2newNocold.csv'
old_list_nocold = list(item_dict.keys())
new_list_nocold = list(item_dict.values())
old2new_nocold_dict = {'old':old_list_nocold, 'new':new_list_nocold}
data_old_new_no = pd.DataFrame(old2new_nocold_dict)
data_old_new_no.to_csv(old2new_nocold_path)

# tes_seqs_nocold, tes_pri_nocold, tes_seqs_cold, tes_pri_cold  = obtian_tes_cold()
# # save item_dict:(old_itemID: new_itemID) cold
# old2new_cold_path = './dict/' + datasets_name + '/old2newCold.csv'
# old_list_cold = list(item_dict.keys())
# new_list_cold = list(item_dict.values())
# old2new_cold_dict = {'old':old_list_cold, 'new':new_list_cold}
# data_old_new_cold = pd.DataFrame(old2new_cold_dict)
# data_old_new_cold.to_csv(old2new_cold_path)

tr_seqs, tr_pri, tr_cat, tr_labs = process_seqs_no(tra_seqs, tra_pri, tra_cat)
te_seqs, te_pri, te_cat, te_labs = process_seqs_no(tes_seqs, tes_pri, tes_cat)

# 冷启动情况下，两个数据集
# te_seqs_nocold, te_pri_nocold, te_labs_nocold = process_seqs_no(tes_seqs_nocold, tes_pri_nocold)
# te_seqs_cold, te_pri_cold, te_labs_cold = process_seqs_no(tes_seqs_cold, tes_pri_cold)

print('train sequence: ',tr_seqs[:5])
print('train price: ',tr_pri[:5])
print('train category: ',tr_cat[:5])
print('train lab: ',tr_labs[:5])

# construct all matrics whose shape is similar as session-items [[],[]]
#  Cell_Phones_and_Accessories_brand 各类型数据量
# def tomatrix(all_seqs, all_pri):

#     price_item_dict = {}
#     price_item = []

#     # price-item-dict -> {price_id:[1, 3, 4]}

#     for s_seq, p_seq in zip(all_seqs, all_pri):
#         for i_temp, p_temp in zip(s_seq, p_seq):
#             if p_temp not in price_item_dict:
# #                 print('price_new: ',p_temp)
#                 price_item_dict[p_temp] = []
#             price_item_dict[p_temp].append(i_temp)

#     price_item_dict = dict(sorted(price_item_dict.items()))
#     print("#price",len(price_item_dict))
    
#     price_item = list(price_item_dict.values())
#     return price_item

# def data_masks(all_sessions):
#     indptr, indices, data = [], [], []
#     indptr.append(0)
#     for j in range(len(all_sessions)):
#         session = np.unique(all_sessions[j]) #统计session中不同item，去重，并按照item_id排序
#         length = len(session)
#         s = indptr[-1]
#         indptr.append((s + length))
#         for i in range(length):
#             indices.append(session[i]-1)
#             data.append(1)
#     # indptr:session长度累加和; indices:item_id 减1, 由每个session内item组成; data:item在session内的权重，全部为1.
#     results = (data, indices, indptr)
#     # 10000 * 6558 #sessions * #items H in paper 稀疏矩阵存储
#     return results

# tra_pi = tomatrix(tra_seqs+tes_seqs, tra_pri+tes_pri)
# # tes_pi, tes_pb, tes_pc, tes_bi, tes_bc, tes_ci = tomatrix(te_seqs, te_pri, te_cat, te_bra)

# # 冷启动下两个测试集，一个不包含cold_item 一个包含cold_item
# tra_pi_nocold = tomatrix(tra_seqs+te_seqs_nocold, tra_pri+te_pri_nocold)
# tra_pi_cold = tomatrix(tra_seqs+te_seqs_cold, tra_pri+te_pri_cold)

# # tra = (tr_seqs, tr_pri, data_masks(tr_seqs), data_masks(tr_pri), data_masks(tra_pi), tr_labs)
# # tes = (te_seqs, te_pri, data_masks(te_seqs), data_masks(te_pri), data_masks(tra_pi), te_labs)
# # 暂时不加tr_pri 即 session_id: [p1, p2, p3] 原代码如上两句
# tra = (tr_seqs, tr_pri, data_masks(tr_seqs), data_masks(tra_pi), tr_labs)
# tes = (te_seqs, te_pri, data_masks(te_seqs), data_masks(tra_pi), te_labs)
# # 冷启动下两个测试集，一个不包含cold_item 一个包含cold_item
# tes_nocold = (te_seqs_nocold, te_pri_nocold, data_masks(te_seqs_nocold), data_masks(tra_pi_nocold), te_labs_nocold)
# tes_cold = (te_seqs_cold, te_pri_cold, data_masks(te_seqs_cold), data_masks(tra_pi_cold), te_labs_cold)




# 增加 all_item_price_list & all_item_rating_list

ip_id_list = list(item_price.keys())
ip_price_list = list(item_price.values())
id_price_dict = {'id':ip_id_list, 'price':ip_price_list}
id_price_dict = pd.DataFrame(id_price_dict)
id_price_dict = id_price_dict.reset_index()[['id', 'price']]

id_price_dict.sort_values(by=["id"],inplace=True,ascending=[True])
price_list_save = id_price_dict['price'].tolist()



ic_id_list = list(item_cate.keys())
ic_cate_list = list(item_cate.values())
id_cate_dict = {'id':ic_id_list, 'cate':ic_cate_list}
id_cate_dict = pd.DataFrame(id_cate_dict)
id_cate_dict = id_cate_dict.reset_index()[['id', 'cate']]

id_cate_dict.sort_values(by=["id"],inplace=True,ascending=[True])
cate_list_save = id_cate_dict['cate'].tolist()



tra = (tr_seqs, tr_pri, tr_cat, price_list_save, cate_list_save, tr_labs)
tes = (te_seqs, te_pri, te_cat, price_list_save, cate_list_save, te_labs)

# print(len(tra[3]))
all_train = 0
all_tes = 0
# all_tes_nocold = 0
# all_tes_cold = 0
for seq in tra_seqs:
    all_train += len(seq)
for seq in tes_seqs:
    all_tes += len(seq)
# for seq in tes_seqs_nocold:
#     all_tes_nocold += len(seq)
# for seq in tes_seqs_cold:
#     all_tes_cold += len(seq)
print('#train_interactions: ',all_train)
print('#test_interactions: ',all_tes)
# print('#nocold_test_interactions: ',all_tes_nocold)
# print('#cold_test_interactions: ',all_tes_cold)

print('#train_session: ',(len(tra_seqs)))
print('#test_session: ',(len(tes_seqs)))
# print('#nocold_test_session: ',(len(tes_seqs_nocold)))
# print('#cold_test_session: ',(len(tes_seqs_cold)))

print('sequence average length: ', (all_train+all_tes)/(len(tra_seqs) + len(tes_seqs) * 1.0))
# print('cold sequence average length: ', (all_train+all_tes_nocold+all_tes_cold)/(len(tra_seqs) + len(tes_seqs_nocold) +len(tes_seqs_cold) * 1.0))
train_data_path = './datasets/' + datasets_name + str(price_num) + 'logistic'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_normal_test = train_data_path + "/test.txt"
# path_nocold_test = train_data_path + "/nocoldtest.txt"
# path_cold_test = train_data_path + "/coldtest.txt"
# path_data_pc = train_data_path + "/itemlistPCB.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_normal_test, 'wb'))
# pickle.dump(tes_nocold, open(path_nocold_test, 'wb'))
# pickle.dump(tes_cold, open(path_cold_test, 'wb'))
# pickle.dump(price_cate_list, open(path_data_pc, 'wb'))


# get asin item_id 获得asin list 有顺序, 给处理 img及text
asin2old_path = 'dict/' + datasets_name +'/asin2itemID.csv'
old2new_path = 'dict/' + datasets_name +'/old2newNocold.csv'
# id_img_emb = 'dict/' + datasets_name +'/id_imgEmbedding.csv'
# id_text_emb = 'dict/' + datasets_name +'/id_textEmbedding.csv'

# text_matrix_path = 'dict/' + datasets_name +'/text_matrix.npy'
# image_matrix_path = 'dict/' + datasets_name +'/image_matrix.npy'


asin2old = pd.read_csv(asin2old_path)
asin2old = asin2old.rename(columns={ 'itemID':'old'})
asin2old = asin2old[['asin', 'old']]


old2new = pd.read_csv(old2new_path)
old2new = old2new[['old', 'new']]


asin2new = pd.merge(asin2old, old2new, how='left', on = 'old')


asin2new = asin2new.dropna(axis=0)
asin2ItemID = asin2new.reset_index()[['asin', 'new']]

asin2ItemID.sort_values(by=["new"],inplace=True,ascending=[True])


asin_list = asin2ItemID['asin'].tolist()
# new_list = asin2ItemID['new'].tolist()
asin_list_save = train_data_path + '/asinlist.npy'
np.save(asin_list_save, asin_list)



print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("data formulation: id_seq, price_seq, cate_seq, price_list, cate_list, labs")
print("done")



#train_session 70260
#train_items 8614
#train_price 10
#train_category 48
#test_session:  7766
no data augment
no data augment
train sequence:  [[1], [3], [5], [7, 8], [10, 11]]
train price:  [[1], [3], [3], [4, 1], [3, 1]]
train category:  [[1], [2], [3], [5, 5], [6, 2]]
train lab:  [2, 4, 6, 9, 12]
#train_interactions:  176784
#test_interactions:  19592
#train_session:  70260
#test_session:  7766
sequence average length:  2.5168020916104887
dataset:  Cell_Phones_and_Accessories
2022-09-23 16:34:44
data formulation: id_seq, price_seq, cate_seq, price_list, cate_list, labs
done


## test length features

In [1]:
import os
import time
import pickle
import pandas as pd
import numpy as np

datasets_name = 'Cell_Phones_and_Accessories'
data_path = 'dict/' + datasets_name +'/session_data.csv'

price_num = '100'

data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID', 'priceLevel', 'category']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_inter_num = item_inter_num.reset_index()[['sessionID', 'itemID']]
# item_num=item_inter_num.rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>4]
# data = data[['sessionID', 'itemID', 'priceLevel']]

# dict (sessionID:[itemID,itemID])
sess_all = {}
# dict (sessionID:[priceLevel, priceLevel])
price_all = {}



# dict (sessionID:[cate, cate])
cate_all = {}
# dict (sessionID:[brand, brand])
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    price = row['priceLevel']
    cate = row['category']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        price_all[sess_id].append(price)
        cate_all[sess_id].append(cate)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        price_all[sess_id] = []
        price_all[sess_id].append(price)
        cate_all[sess_id] = []
        cate_all[sess_id].append(cate)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() 
tra_price = dict()# dict(session_id:[price,price])
tes_price = dict()
tra_cate = dict()# dict(session_id:[cate,cate])
tes_cate = dict()
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    all_price = price_all[sess_temp]
    all_cate = cate_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
        all_price = all_price[:20]
        all_cate = all_cate[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
        tra_price[sess_temp] = all_price
        tra_cate[sess_temp] = all_cate
    else:
        tes_sess[sess_temp] = all_seqs
        tes_price[sess_temp] = all_price
        tes_cate[sess_temp] = all_cate

# the number of item & price, renumber
# item_ctr = 1
# price_ctr = 1
# cate_ctr = 1

item_dict = {} #dict(old_itemID: new_itemID)
cate_dict = {} #dict(old_cate: new_cate)
price_dict = {} #dict(old_price: new_price)


# item_id: price_level, item_id: rating
item_price = {} #dict[new_itemID: priceLevel]
item_cate = {} #dict[new_itemID: cate]




# tra_sess tra_price tra_cate
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
#     global item_ctr
#     print(item_ctr)
    train_seqs = []
    train_price = []
    train_cate = []
#     train_brand = []
    item_ctr = 1
    price_ctr = 1
    cate_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        price_seq = tra_price[s]
        cate_seq = tra_cate[s]
        outseq = []
        pri_outseq = []
#         rating_outseq = []
        cate_outseq = []
        for (i, j_pri, w_cate) in zip(seq, price_seq, cate_seq):
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
            if j_pri in price_dict:
                pri_outseq += [price_dict[j_pri]]
            else:
                pri_outseq += [price_ctr]
                price_dict[j_pri] = price_ctr
                price_ctr += 1
            if w_cate in cate_dict:
                cate_outseq += [cate_dict[w_cate]]
            else:
                cate_outseq += [cate_ctr]
                cate_dict[w_cate] = cate_ctr
                cate_ctr += 1
            if item_dict[i] not in item_price:
                item_price[item_dict[i]] = price_dict[j_pri]
                item_cate[item_dict[i]] = cate_dict[w_cate]
        if len(outseq) < 2:  # Doesn't occur
            print('session length is 1')
            continue
        train_seqs += [outseq]
        train_price += [pri_outseq]
        train_cate += [cate_outseq]
    print("#train_session",len(train_seqs))
    print("#train_items",item_ctr-1)
    print("#train_price",price_ctr-1)
    print("#train_category",cate_ctr-1)
    return train_seqs, train_price, train_cate


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    test_price = []
    test_cate = []
    for s in tes_sess:
        outseq = []
        out_price = []
        out_rating = []
        out_cate = []
        for i, j, w in zip(tes_sess[s], tes_price[s], tes_cate[s]):
            if i in item_dict:
                outseq += [item_dict[i]]
                out_price += [price_dict[j]]
                out_cate += [cate_dict[w]]
        if len(outseq) < 2:
            print('obtain test session length is 1')
            continue
        test_seqs += [outseq]
        test_price += [out_price]
        test_cate += [out_cate]
    print("#test_session: ",len(test_seqs))
    return test_seqs, test_price, test_cate

# Convert test sessions to sequences, renumbering items that do not appear in training set


# 数据增强
def process_seqs(iseqs, iprice, icate):
    out_seqs = []
    out_price = []
    out_cate = []
    labs = []
    max_length = 19
    for seq, pri, rat, cat in zip(iseqs, iprice, icate):
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
            out_price += [pri[:-i]]
            out_cate += [cat[:-i]]
    return out_seqs, out_price, out_cate, labs

# 无数据增强
def process_seqs_no(iseqs, iprice, icate):
    print("no data augment")
    out_seqs = []
    out_price = []
    out_cate = []
    labs = []
    max_length = 19
    for seq, pri, cat in zip(iseqs, iprice, icate):     
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
        out_price += [pri[:-1]]
        out_cate += [cat[:-1]]
    return out_seqs, out_price, out_cate, labs

tra_seqs, tra_pri, tra_cat  = obtian_tra()
tes_seqs, tes_pri, tes_cat  = obtian_tes()
# save item_dict:(old_itemID: new_itemID) nocold
old2new_nocold_path = './dict/' + datasets_name + '/old2newNocold.csv'
old_list_nocold = list(item_dict.keys())
new_list_nocold = list(item_dict.values())
old2new_nocold_dict = {'old':old_list_nocold, 'new':new_list_nocold}
data_old_new_no = pd.DataFrame(old2new_nocold_dict)
data_old_new_no.to_csv(old2new_nocold_path)



tr_seqs, tr_pri, tr_cat, tr_labs = process_seqs_no(tra_seqs, tra_pri, tra_cat)
te_seqs, te_pri, te_cat, te_labs = process_seqs_no(tes_seqs, tes_pri, tes_cat)

te_seqs1 = []
te_seqs2 = []
te_seqs3 = []
te_seqs4 = []
te_seqs5 = []

te_pri1 = []
te_pri2 = []
te_pri3 = []
te_pri4 = []
te_pri5 = []

te_cat1 = []
te_cat2 = []
te_cat3 = []
te_cat4 = []
te_cat5 = []

te_labs1 = []
te_labs2 = []
te_labs3 = []
te_labs4 = []
te_labs5 = []

num_1 = 0
num_2 = 0
num_3 = 0
num_4 = 0
num_5 = 0

for i_sess, j_pri, k_cat, l_lab in zip(te_seqs, te_pri, te_cat, te_labs):
    if len(i_sess) == 1:
        te_seqs1 += [i_sess]
        te_pri1 += [j_pri]
        te_cat1 += [k_cat]
        te_labs1 += [l_lab]
        num_1 += 1
    elif len(i_sess) == 2:
        te_seqs2 += [i_sess]
        te_pri2 += [j_pri]
        te_cat2 += [k_cat]
        te_labs2 += [l_lab]
        num_2 += 1
    elif len(i_sess) == 3:
        te_seqs3 += [i_sess]
        te_pri3 += [j_pri]
        te_cat3 += [k_cat]
        te_labs3 += [l_lab]
        num_3 += 1
    elif len(i_sess) == 4:
        te_seqs4 += [i_sess]
        te_pri4 += [j_pri]
        te_cat4 += [k_cat]
        te_labs4 += [l_lab]
        num_4 += 1
    else:
        te_seqs5 += [i_sess]
        te_pri5 += [j_pri]
        te_cat5 += [k_cat]
        te_labs5 += [l_lab]
        num_5 += 1

# 冷启动情况下，两个数据集
# te_seqs_nocold, te_pri_nocold, te_labs_nocold = process_seqs_no(tes_seqs_nocold, tes_pri_nocold)
# te_seqs_cold, te_pri_cold, te_labs_cold = process_seqs_no(tes_seqs_cold, tes_pri_cold)

print('train sequence: ',tr_seqs[:5])
print('train price: ',tr_pri[:5])
print('train category: ',tr_cat[:5])
print('train lab: ',tr_labs[:5])






# 增加 all_item_price_list & all_item_rating_list

ip_id_list = list(item_price.keys())
ip_price_list = list(item_price.values())
id_price_dict = {'id':ip_id_list, 'price':ip_price_list}
id_price_dict = pd.DataFrame(id_price_dict)
id_price_dict = id_price_dict.reset_index()[['id', 'price']]

id_price_dict.sort_values(by=["id"],inplace=True,ascending=[True])
price_list_save = id_price_dict['price'].tolist()



ic_id_list = list(item_cate.keys())
ic_cate_list = list(item_cate.values())
id_cate_dict = {'id':ic_id_list, 'cate':ic_cate_list}
id_cate_dict = pd.DataFrame(id_cate_dict)
id_cate_dict = id_cate_dict.reset_index()[['id', 'cate']]

id_cate_dict.sort_values(by=["id"],inplace=True,ascending=[True])
cate_list_save = id_cate_dict['cate'].tolist()



tra = (tr_seqs, tr_pri, tr_cat, price_list_save, cate_list_save, tr_labs)
tes = (te_seqs, te_pri, te_cat, price_list_save, cate_list_save, te_labs)

tes1 = (te_seqs1, te_pri1, te_cat1, price_list_save, cate_list_save, te_labs1)
tes2 = (te_seqs2, te_pri2, te_cat2, price_list_save, cate_list_save, te_labs2)
tes3 = (te_seqs3, te_pri3, te_cat3, price_list_save, cate_list_save, te_labs3)
tes4 = (te_seqs4, te_pri4, te_cat4, price_list_save, cate_list_save, te_labs4)
tes5 = (te_seqs5, te_pri5, te_cat5, price_list_save, cate_list_save, te_labs5)

# print(len(tra[3]))
all_train = 0
all_tes = 0
# all_tes_nocold = 0
# all_tes_cold = 0
for seq in tra_seqs:
    all_train += len(seq)
for seq in tes_seqs:
    all_tes += len(seq)
# for seq in tes_seqs_nocold:
#     all_tes_nocold += len(seq)
# for seq in tes_seqs_cold:
#     all_tes_cold += len(seq)
print('#train_interactions: ',all_train)
print('#test_interactions: ',all_tes)
# print('#nocold_test_interactions: ',all_tes_nocold)
# print('#cold_test_interactions: ',all_tes_cold)

print('#train_session: ',(len(tra_seqs)))
print('#test_session: ',(len(tes_seqs)))
print('length_1:',str(num_1))
print('length_2:',str(num_2))
print('length_3:',str(num_3))
print('length_4:',str(num_4))
print('length_5:',str(num_5))
# print('#nocold_test_session: ',(len(tes_seqs_nocold)))
# print('#cold_test_session: ',(len(tes_seqs_cold)))

print('sequence average length: ', (all_train+all_tes)/(len(tra_seqs) + len(tes_seqs) * 1.0))
# print('cold sequence average length: ', (all_train+all_tes_nocold+all_tes_cold)/(len(tra_seqs) + len(tes_seqs_nocold) +len(tes_seqs_cold) * 1.0))
train_data_path = './datasets/' + datasets_name + str(price_num) + 'length'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_normal_test = train_data_path + "/test.txt"
path_test1 = train_data_path + "/test1.txt"
path_test2 = train_data_path + "/test2.txt"
path_test3 = train_data_path + "/test3.txt"
path_test4 = train_data_path + "/test4.txt"
path_test5 = train_data_path + "/test5.txt"
# path_nocold_test = train_data_path + "/nocoldtest.txt"
# path_cold_test = train_data_path + "/coldtest.txt"
# path_data_pc = train_data_path + "/itemlistPCB.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_normal_test, 'wb'))
pickle.dump(tes1, open(path_test1, 'wb'))
pickle.dump(tes2, open(path_test2, 'wb'))
pickle.dump(tes3, open(path_test3, 'wb'))
pickle.dump(tes4, open(path_test4, 'wb'))
pickle.dump(tes5, open(path_test5, 'wb'))

# pickle.dump(tes_nocold, open(path_nocold_test, 'wb'))
# pickle.dump(tes_cold, open(path_cold_test, 'wb'))
# pickle.dump(price_cate_list, open(path_data_pc, 'wb'))


# get asin item_id 获得asin list 有顺序, 给处理 img及text
asin2old_path = 'dict/' + datasets_name +'/asin2itemID.csv'
old2new_path = 'dict/' + datasets_name +'/old2newNocold.csv'
# id_img_emb = 'dict/' + datasets_name +'/id_imgEmbedding.csv'
# id_text_emb = 'dict/' + datasets_name +'/id_textEmbedding.csv'

# text_matrix_path = 'dict/' + datasets_name +'/text_matrix.npy'
# image_matrix_path = 'dict/' + datasets_name +'/image_matrix.npy'


asin2old = pd.read_csv(asin2old_path)
asin2old = asin2old.rename(columns={ 'itemID':'old'})
asin2old = asin2old[['asin', 'old']]


old2new = pd.read_csv(old2new_path)
old2new = old2new[['old', 'new']]


asin2new = pd.merge(asin2old, old2new, how='left', on = 'old')


asin2new = asin2new.dropna(axis=0)
asin2ItemID = asin2new.reset_index()[['asin', 'new']]

asin2ItemID.sort_values(by=["new"],inplace=True,ascending=[True])


asin_list = asin2ItemID['asin'].tolist()
# new_list = asin2ItemID['new'].tolist()
asin_list_save = train_data_path + '/asinlist.npy'
np.save(asin_list_save, asin_list)



print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("data formulation: id_seq, price_seq, cate_seq, price_list, cate_list, labs")
print("done")



#train_session 70260
#train_items 8614
#train_price 10
#train_category 48
#test_session:  7766
no data augment
no data augment
train sequence:  [[1], [3], [5], [7, 8], [10, 11]]
train price:  [[1], [3], [3], [4, 1], [3, 1]]
train category:  [[1], [2], [3], [5, 5], [6, 2]]
train lab:  [2, 4, 6, 9, 12]
#train_interactions:  176784
#test_interactions:  19592
#train_session:  70260
#test_session:  7766
length_1: 5351
length_2: 1496
length_3: 521
length_4: 240
length_5: 158
sequence average length:  2.5168020916104887
dataset:  Cell_Phones_and_Accessories
2022-09-28 15:34:59
data formulation: id_seq, price_seq, cate_seq, price_list, cate_list, labs
done


### NARM, S-POP, SR-GNN, DHCN 使用相同的 train&test

In [2]:
import os
import pickle
import time
import pandas as pd
import numpy as np


datasets_name = 'Office_Products'
data_path = 'dict/' + datasets_name +'/session_data.csv'
# data_path = 'digineticaBuy/digineticaBuy_data.csv'

# datasets_name = '2019-Oct'
# price_num = 5

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'
# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'



data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID']]

# dict (sessionID:[itemID,itemID])
sess_all = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs

item_dict = {}#dict(old_itemID: new_itemID)
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
    item_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
    print("#train_items",item_ctr)    
    return train_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
    return test_seqs

# 数据增强
def process_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

# 无数据增强
def process_seqs_no(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
    return out_seqs, labs


tra_seqs = obtian_tra()
tes_seqs = obtian_tes()

tr_seqs, tr_labs = process_seqs_no(tra_seqs)
te_seqs, te_labs = process_seqs_no(tes_seqs)

tra = (tr_seqs, tr_labs)
tes = (te_seqs, te_labs)
print("train sequence: ", len(tr_seqs))
print("test sequence: ", len(te_seqs))
print("train sample: ", tr_seqs[:3], tr_labs[:3])
print("test sample: ", te_seqs[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('sequence average length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

train_data_path = 'NARM/' + datasets_name

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))
print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

#train_items 9151
train sequence:  89983
test sequence:  10031
train sample:  [[1, 2, 3], [4], [6]] [3, 5, 7]
test sample:  [[3686], [1102, 5865], [7243, 1682]] [2915, 2429, 1870]
sequence average length:  2.866678664986902
dataset:  Office_Products
2022-09-12 08:52:57
done


### MGS

In [1]:
import os
import pickle
import time
import pandas as pd
import numpy as np
import json


datasets_name = 'Office_Products'
data_path = 'dict/' + datasets_name +'/session_data.csv'
# data_path = 'digineticaBuy/digineticaBuy_data.csv'

# datasets_name = '2019-Oct'
# price_num = 5

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'
# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'



data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID', 'category']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID']]

# dict (sessionID:[itemID,itemID])
sess_all = {}
# dict (sessionID:[cate, cate])
cate_all = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    cate = row['category']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        cate_all[sess_id].append(cate)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        cate_all[sess_id] = []
        cate_all[sess_id].append(cate)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
tra_cate = dict()# dict(session_id:[cate,cate])
tes_cate = dict()
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    all_cate = cate_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
        all_cate = all_cate[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
        tra_cate[sess_temp] = all_cate
    else:
        tes_sess[sess_temp] = all_seqs
        tes_cate[sess_temp] = all_cate

item_dict = {}#dict(old_itemID: new_itemID)
cate_dict = {} #dict(old_cate: new_cate)


# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
    train_cate = []
    item_ctr = 1
    cate_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        cate_seq = tra_cate[s]
        outseq = []
        cate_outseq = []
        for (i, w_cate) in zip(seq, cate_seq):
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
            if w_cate in cate_dict:
                cate_outseq += [cate_dict[w_cate]]
            else:
                cate_outseq += [cate_ctr]
                cate_dict[w_cate] = cate_ctr
                cate_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
        train_cate += [cate_outseq]
    print("#train_items",item_ctr)
    print("#train_category",cate_ctr)
    return train_seqs, train_cate


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
    return test_seqs

# 数据增强
def process_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

# 无数据增强
def process_seqs_no(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
    return out_seqs, labs


tra_seqs,train_cate = obtian_tra()
tes_seqs = obtian_tes()

#  形成 json about cate
item_cate = {} #dict[new_itemID: new_cate]
cate_itemlist = {} #dict[new_cate: [new_itemID,new_itemID]]
for (item_seq, cate_seq) in zip(tra_seqs, train_cate):
    for (x_item, y_cate) in zip(item_seq,cate_seq):
        if x_item not in item_cate:
            item_cate[x_item] = y_cate
        if y_cate not in cate_itemlist:
            cate_itemlist[y_cate] = []
            cate_itemlist[y_cate].append(x_item)
        else:
            if x_item not in cate_itemlist[y_cate]:
                cate_itemlist[y_cate].append(x_item)

dict_results =dict()
for x in item_cate:
    x_str = x
    if x_str not in dict_results:
        dict_results[x_str] = {"category":item_cate[x_str], "same_cate":cate_itemlist[item_cate[x_str]]}

tr_seqs, tr_labs = process_seqs_no(tra_seqs)
te_seqs, te_labs = process_seqs_no(tes_seqs)

tra = (tr_seqs, tr_labs)
tes = (te_seqs, te_labs)
print("train sequence: ", len(tr_seqs))
print("test sequence: ", len(te_seqs))
print("train sample: ", tr_seqs[:3], tr_labs[:3])
print("test sample: ", te_seqs[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('sequence average length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

train_data_path = 'MGS/' + datasets_name

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"
path_json = train_data_path + "/product_attributes.json"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))
with open(path_json, "w") as file_object:
    json.dump(dict_results, file_object)
print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("model:","MGS")
print("done")

#train_items 9151
#train_category 528
train sequence:  89983
test sequence:  10031
train sample:  [[1, 2, 3], [4], [6]] [3, 5, 7]
test sample:  [[3686], [1102, 5865], [7243, 1682]] [2915, 2429, 1870]
sequence average length:  2.866678664986902
dataset:  Office_Products
2022-09-12 08:53:35
model: MGS
done


### UniSRec

In [2]:
import os
import pickle
import time
import pandas as pd
import numpy as np


datasets_name = 'Office_Products'
data_path = 'dict/' + datasets_name +'/session_data.csv'
# data_path = 'digineticaBuy/digineticaBuy_data.csv'

# datasets_name = '2019-Oct'
# price_num = 5

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'
# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'



data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID']]

# dict (sessionID:[itemID,itemID])
sess_all = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs

item_dict = {}#dict(old_itemID: new_itemID)
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
#   UniSRec item 0 序
    item_ctr = 0
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
    print("#train_items",item_ctr)    
    return train_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
    return test_seqs

# 数据增强
def process_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

# 无数据增强
def process_seqs_no(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
    return out_seqs, labs


tra_seqs = obtian_tra()
tes_seqs = obtian_tes()

tr_seqs, tr_labs = process_seqs_no(tra_seqs)
te_seqs, te_labs = process_seqs_no(tes_seqs)

# tra = (tr_seqs, tr_labs)
# tes = (te_seqs, te_labs)
print("train sequence: ", len(tr_seqs))
print("test sequence: ", len(te_seqs))
print("train sample: ", tr_seqs[:3], tr_labs[:3])
print("test sample: ", te_seqs[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('sequence average length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

train_data_path = 'UniSRec/' + datasets_name + '/'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
    
path_data_train = train_data_path + f'{datasets_name}.train.inter'
path_data_valid = train_data_path + f'{datasets_name}.valid.inter'
path_data_test = train_data_path + f'{datasets_name}.test.inter'

split_valid = int(len(tr_labs)/9*7)
train_sess = tr_seqs[:split_valid]
train_labs = tr_labs[:split_valid]
valid_sess = tr_seqs[split_valid:]
valid_labs = tr_labs[split_valid:]

with open(path_data_train, 'w') as file:
    file.write('user_id:token\titem_id_list:token_seq\titem_id:token\n')
    for index, sess, lab in zip(range(len(train_labs)),train_sess, train_labs):
        i_str = str(index)
        s_str = [str(x) for x in sess]
        l_str = str(lab)
        file.write(f'{i_str}\t{" ".join(s_str)}\t{l_str}\n')
                   
with open(path_data_valid, 'w') as file:
    file.write('user_id:token\titem_id_list:token_seq\titem_id:token\n')
    for index, sess, lab in zip(range(len(valid_labs)),valid_sess, valid_labs):
        i_str = str(index)
        s_str = [str(x) for x in sess]
        l_str = str(lab)
        file.write(f'{i_str}\t{" ".join(s_str)}\t{l_str}\n')
                   
with open(path_data_test, 'w') as file:
    file.write('user_id:token\titem_id_list:token_seq\titem_id:token\n')
    for index, sess, lab in zip(range(len(te_labs)),te_seqs, te_labs):
        i_str = str(index)
        s_str = [str(x) for x in sess]
        l_str = str(lab)
        file.write(f'{i_str}\t{" ".join(s_str)}\t{l_str}\n')
                   
print("dataset: ", datasets_name)
print("model: UniSRec")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

#train_items 9150
train sequence:  89983
test sequence:  10031
train sample:  [[0, 1, 2], [3], [5]] [2, 4, 6]
test sample:  [[3685], [1101, 5864], [7242, 1681]] [2914, 2428, 1869]
sequence average length:  2.866678664986902
dataset:  Office_Products
model: UniSRec
2022-09-12 08:55:19
done


## SSLREC - senti

In [ ]:
import os
import pickle
import time
import pandas as pd
import numpy as np


datasets_name = 'Cell_Phones_and_Accessories'
data_path = 'dict/' + datasets_name +'/session_data.csv'
# data_path = 'digineticaBuy/digineticaBuy_data.csv'

# datasets_name = '2019-Oct'
# price_num = 5

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'
# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'



data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]

item_count = 0
user_count = 0
inter_count = 0

# dict (sessionID:[itemID,itemID])
sess_all = {}
#  dict {old_itemID: title}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
        
item_num = 1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[u1, u2],[x1, x2, x3], [label...]
tra = [[],[],[]]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(sess_temp)
    tra[1].append(new_seq[:-1])
#     预测时考虑last item
    tra[2].append(new_seq[-1])
    
item_count =  item_num-1
user_count = len(tra[0])

# test_set [[x_id ...],[label...]
tes = [[],[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes_seq_num += 1
        tes[0].append(sess_temp)
        tes[1].append(new_seq[:-1])
    #     预测时考虑last item
        tes[2].append(new_seq[-1])
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num


print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)

train_data_path = 'SSLRecFine/' + datasets_name + 'Fine/'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)


path_data_train = train_data_path + 'train.tsv'
# path_data_valid = train_data_path + f'{datasets_name_c}.valid.inter'
path_data_test = train_data_path + 'test.tsv'
# train_sessid_list = range(len(tr_labs))
with open(path_data_train, 'w') as file:
    file.write('session_id\t item_id_seq\t item_id \n')
    for index, sess, lab in zip(tra[0],tra[1], tra[2]):
        i_str = str(index-1)
        s_str = [str(x) for x in sess]
        l_str = str(lab)
        file.write(f'{i_str}\t{" ".join(s_str)}\t{l_str}\n')


with open(path_data_test, 'w') as file:
    file.write('session_id\t item_id_seq\t item_id \n')
    for index, sess, lab in zip(tes[0],tes[1], tes[2]):
        i_str = str(index-1)
        s_str = [str(x) for x in sess]
        l_str = str(lab)
        file.write(f'{i_str}\t{" ".join(s_str)}\t{l_str}\n')


# train_sessid_list = range(len(tr_labs))
# train_seq_list = tr_seqs
# train_lab_list = tr_labs
# train_pd = pd.DataFrame({'session_id':train_sessid_list, 'item_id_seq':train_seq_list, 'item_id':train_lab_list})

# test_sessid_list = range(len(te_labs))
# test_seq_list = te_seqs
# test_lab_list = te_labs
# test_pd = pd.DataFrame({'session_id':test_sessid_list, 'item_id_seq':test_seq_list, 'item_id':test_lab_list})

# train_pd.to_csv(path_data_train, sep = '\t', index=False)
# test_pd.to_csv(path_data_test, sep = '\t', index=False)

print("dataset: ", datasets_name)
print("model: SSLRec")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

In [2]:
list(range(5))

[0, 1, 2, 3, 4]

### MML

In [3]:
import os
import pickle
import time
import pandas as pd
import numpy as np


datasets_name = 'Cell_Phones_and_Accessories'
data_path = 'dict/' + datasets_name +'/session_data.csv'
# data_path = 'digineticaBuy/digineticaBuy_data.csv'

# datasets_name = '2019-Oct'
# price_num = 5

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'
# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'



data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID']]

# dict (sessionID:[itemID,itemID])
sess_all = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs

item_dict = {}#dict(old_itemID: new_itemID)
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
#   MML item 1 序
    item_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
    print("#train_items",item_ctr)    
    return train_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
    return test_seqs

# 数据增强
def process_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

# 无数据增强
def process_seqs_no(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
    return out_seqs, labs


tra_seqs = obtian_tra()
tes_seqs = obtian_tes()

mml_seqs = tra_seqs + tes_seqs

# tra = (tr_seqs, tr_labs)
# tes = (te_seqs, te_labs)

all = 0

for seq in mml_seqs:
    all += len(seq)

print('sequence average length: ', all/(len(mml_seqs) * 1.0))
print('split sess id: ',str(len(tra_seqs)))

train_data_path = 'MML/' + datasets_name + '/'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
    
path_data_train = train_data_path + f'{datasets_name}.inter'


with open(path_data_train, 'w') as file:
    file.write('user_id:token\titem_id:token\trating:float\ttimestamp:float\n')
    for index, sess in zip(range(len(mml_seqs)),mml_seqs):
        i_str = str(index+1)
        for item_str in sess:
            item_str = str(item_str)
            file.write(f'{i_str}\t{item_str}\t{i_str}\t{i_str}\n')
                   
                   
print("dataset: ", datasets_name)
print("model: MML")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

#train_items 8615
sequence average length:  2.5168020916104887
split sess id:  70260
dataset:  Cell_Phones_and_Accessories
model: MML
2023-06-27 18:35:50
done


### COTREC

In [3]:
import os
import pickle
import time
import pandas as pd
import numpy as np

datasets_name = 'Office_Products'
data_path = 'dict/' + datasets_name +'/session_data.csv'

# datasets_name = '2019-Oct'
# price_num = 'logistic'

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'

# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'

# data_path = 'data_time_interval/' + datasets_name + '_data.csv'
data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID']]

# dict (sessionID:[itemID,itemID])
sess_all = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs

item_dict = {}#dict(old_itemID: new_itemID)
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
    item_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
    print("#train_items",item_ctr)    
    return train_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
    return test_seqs

# 数据增强
def process_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

# 无数据增强
def process_seqs_no(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
    return out_seqs, labs

tra_seqs = obtian_tra()
tes_seqs = obtian_tes()

all_train_seqs = tra_seqs + tes_seqs

tr_seqs, tr_labs = process_seqs_no(tra_seqs)
te_seqs, te_labs = process_seqs_no(tes_seqs)

tra = (tr_seqs, tr_labs)
tes = (te_seqs, te_labs)
print("train sequence: ", len(tr_seqs))
print("test sequence: ", len(te_seqs))
print("train sample: ", tr_seqs[:3], tr_labs[:3])
print("test sample: ", te_seqs[:3], te_labs[:3])
print("all train sequences:", all_train_seqs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('sequence average length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

train_data_path = 'COTREC/' + datasets_name

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"
path_data_all = train_data_path+ "/all_train_seq.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))
pickle.dump(all_train_seqs, open(path_data_all, 'wb'))
print("dataset: ", datasets_name)
print("COTREC")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

#train_items 9151
train sequence:  89983
test sequence:  10031
train sample:  [[1, 2, 3], [4], [6]] [3, 5, 7]
test sample:  [[3686], [1102, 5865], [7243, 1682]] [2915, 2429, 1870]
all train sequences: [[1, 2, 3, 3], [4, 5], [6, 7]]
sequence average length:  2.866678664986902
dataset:  Office_Products
COTREC
2022-09-12 08:55:46


## LESSR & MSGIFSR

In [1]:
import os
import time
import pickle
import pandas as pd
import numpy as np

# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'

# datasets_name = '2019-Oct'
# price_num = 'logistic'

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'

datasets_name = 'Office_Products'
# data_path = 'data_time_interval/' + datasets_name +'_data.csv'
data_path = 'dict/' + datasets_name +'/session_data.csv'

data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID']]

# dict (sessionID:[itemID,itemID])
sess_all = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs

item_dict = {}#dict(old_itemID: new_itemID)
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
    item_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
    print("#train_items",item_ctr)    
    return train_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
    return test_seqs


tra_seqs = obtian_tra()
tes_seqs = obtian_tes()

print("train sequence: ", len(tra_seqs))
print("test sequence: ", len(tes_seqs))
print("train sample: ", tra_seqs[:3])
print("test sample: ", tes_seqs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('sequence average length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

# train_data_path = 'LESSR/Cell_Phones_and_Accessories'
train_data_path = 'LESSRFeature/' + datasets_name 

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"
path_item_num = train_data_path + '/num_items.txt'

print('save item_num :', len(item_dict))
pickle.dump(len(item_dict)+1, open(path_item_num, 'wb'))
pickle.dump(tra_seqs, open(path_data_train, 'wb'))
pickle.dump(tes_seqs, open(path_data_test, 'wb'))
print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

#train_items 9151
train sequence:  89983
test sequence:  10031
train sample:  [[1, 2, 3, 3], [4, 5], [6, 7]]
test sample:  [[3686, 2915], [1102, 5865, 2429], [7243, 1682, 1870]]
sequence average length:  2.866678664986902
save item_num : 9150
dataset:  Office_Products
2022-09-12 08:56:19
done


## BERT4Rec

In [2]:
import os
import pickle
import time
import pandas as pd
import numpy as np

# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'

# datasets_name = '2019-Oct'
# price_num = 'logistic'

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'

datasets_name = 'Office_Products'
# data_path = 'data_time_interval/' + datasets_name +'_data.csv'
data_path = 'dict/' + datasets_name +'/session_data.csv'


data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]

# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID']]

# dict (sessionID:[itemID,itemID])
sess_all = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs

item_dict = {}#dict(old_itemID: new_itemID)
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
    sessid_seqs = []
    item_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
        sessid_seqs += [s]
    print("#train_items",item_ctr)    
    return train_seqs, sessid_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    test_sessid_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
        test_sessid_seqs += [s]
    return test_seqs, test_sessid_seqs

# 数据增强
def process_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

# 无数据增强
def process_seqs_no(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
    return out_seqs, labs


tra_seqs, tra_sessid = obtian_tra()
tes_seqs, tes_sessid = obtian_tes()

tr_seqs, tr_labs = process_seqs_no(tra_seqs)
te_seqs, te_labs = process_seqs_no(tes_seqs)

tra = (tra_sessid, tr_seqs, tr_labs)
tes = (tes_sessid, te_seqs, te_labs)
print("train sequence: ", len(tr_seqs))
print("test sequence: ", len(te_seqs))
print("train sample: ", tr_seqs[:3], tr_labs[:3])
print("test sample: ", te_seqs[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('sequence average length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

train_data_path = 'Bert4RecFeature/' + datasets_name

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))
print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

#train_items 9151
train sequence:  89983
test sequence:  10031
train sample:  [[1, 2, 3], [4], [6]] [3, 5, 7]
test sample:  [[3686], [1102, 5865], [7243, 1682]] [2915, 2429, 1870]
sequence average length:  2.866678664986902
dataset:  Office_Products
2022-09-12 08:56:55
done


## SKNN

In [1]:
import os
import pickle
import time
import pandas as pd
import numpy as np

datasets_name = 'Sports_and_Outdoors'
# data_path = 'data_time_interval/' + datasets_name +'_data.csv'
data_path = 'dict/' + datasets_name +'/session_data.csv'

# datasets_name = '2019-Oct'
# price_num = 'logistic'

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'

# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'


print("******SKNN******")

data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID', 'time']]


# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})[['itemID', 'item_num']]
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID', 'time']]
 

# dict (sessionID:[itemID,itemID])
sess_all = {}
sess_date = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    timestamp = row['time']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        sess_date[sess_id] = timestamp
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        sess_date[sess_id] = timestamp


sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs


item_dict = {}#dict(old_itemID: new_itemID)
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
    sessid_seqs = []
    time_seqs = []
    item_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
        sessid_seqs += [s]
        time_seqs += [sess_date[s]]
    print("#train_items",item_ctr)    
    return train_seqs, sessid_seqs, time_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    test_sessid_seqs = []
    test_time_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
        test_sessid_seqs += [s]
        test_time_seqs += [sess_date[s]]
    return test_seqs, test_sessid_seqs, test_time_seqs

# 数据增强
def process_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

# 无数据增强
def process_seqs_no(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
    return out_seqs, labs


tra_seqs, tra_sessid, tra_time = obtian_tra()
tes_seqs, tes_sessid, tes_time = obtian_tes()

tr_seqs, tr_labs = process_seqs_no(tra_seqs)
te_seqs, te_labs = process_seqs_no(tes_seqs)

tra = (tra_sessid, tr_seqs, tr_labs, tra_time)
tes = (tes_sessid, te_seqs, te_labs, tes_time)
print("train sequence: ", len(tr_seqs))
print("test sequence: ", len(te_seqs))
print("train sample: ", tr_seqs[:3], tr_labs[:3])
print("test sample: ", te_seqs[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('sequence average length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

train_data_path = 'SKNNFeature/' + datasets_name

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))
print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

******SKNN******
#train_items 18797
train sequence:  190840
test sequence:  21119
train sample:  [[1, 2], [4], [6, 7, 8]] [3, 5, 9]
test sample:  [[5282, 15212], [900, 2046, 12546, 787], [461, 3225, 461, 3225]] [4461, 4053, 3226]
sequence average length:  2.6727055704169205
dataset:  Sports_and_Outdoors
2022-09-21 13:47:31
done


## SKNN Fine

In [ ]:
import os
import pickle
import time
import pandas as pd
import numpy as np

datasets_name = 'Sports_and_Outdoors'
# data_path = 'data_time_interval/' + datasets_name +'_data.csv'
data_path = 'dict/' + datasets_name +'/session_data.csv'

# datasets_name = '2019-Oct'
# price_num = 'logistic'

# data_path = 'cosmeticsdata/' + datasets_name + '_data.csv'

# datasets_name = 'digineticaBuy'
# data_path = 'digineticaBuy/' + datasets_name + '_data.csv'


print("******SKNN******")

data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID', 'review_time']]


# item_inter_num = pd.DataFrame(data.groupby(data['itemID']).count())
# item_num=item_inter_num.reset_index().rename(columns={'sessionID':'item_num'})[['itemID', 'item_num']]
# data = pd.merge(data, item_num, how='left', on = 'itemID')
# data = data[data['item_num']>9]
# data = data[['sessionID', 'itemID', 'time']]
 

# dict (sessionID:[itemID,itemID])
sess_all = {}
sess_date = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    timestamp = row['review_time']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        sess_date[sess_id] = timestamp
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        sess_date[sess_id] = timestamp
        
# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    time_seq = sess_date[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
        sess_date[sess_temp] = time_seq[-20:]

tra = [[],[],[], []]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

item_count = 0
user_count = 0
inter_count = 0
item_num = 1

for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    t_time_seq = sess_date[sess_temp][:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x in train_seqs_t:
        if x in item_dict:
            new_seq.append(item_dict[x])
        else:
            item_dict[x] = item_num
            newitem_title[item_num] = olditem_title[x]
            new_seq.append(item_num)
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(sess_temp)
    tra[1].append(new_seq[:-1])
#     预测时考虑last item
    tra[2].append(new_seq[-1])
    tra[3].append(t_time_seq)
    
item_count =  item_num-1
user_count = len(tra[0])

# test_set [[x_id ...],[label...]
tes = [[],[],[], []]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    all_time = sess_date[sess_temp]
    if all_seqs[-1] in item_dict:
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes_seq_num += 1
        tes[0].append(sess_temp)
        tes[1].append(new_seq[:-1])
    #     预测时考虑last item
        tes[2].append(new_seq[-1])
        tes[3].append(all_time)
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num


print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)

tra = (tra[0], tra[1], tra[2], tra[3])
tes = (tes[0], tes[1], tes[2], tes[3])

train_data_path = './SKNN/' + datasets_name

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))
print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("SKNN done")

## GRU4Rec with tsv

In [ ]:
import os
import pickle
import time
import pandas as pd
import numpy as np

datasets_name = 'Cell_Phones_and_Accessories'
data_path = 'SKNNFeature/' + datasets_name
path_data_train = data_path + "/train.txt"
path_data_test = data_path + "/test.txt"

csv_train_path = './GRU4RecFeature/' + datasets_name + '/train.tsv'
csv_test_path = './GRU4RecFeature/' + datasets_name + '/test.tsv'

train_data = pickle.load(open(path_data_train, 'rb'))
test_data = pickle.load(open(path_data_test, 'rb'))


def txt2csv(txtdata, data_path):
    listdata = []
    id_seqs = txtdata[0]
    sess_seqs = txtdata[1]
    time_seqs = txtdata[3]
    lab_seqs = txtdata[2]
    for x_id, x_sess, x_time, x_lab in zip(id_seqs, sess_seqs, time_seqs, lab_seqs):
        for y_item in x_sess:
            x_list = []
            x_list.append(x_id)
            x_list.append(y_item)
            x_list.append(x_time)
            listdata.append(x_list)
        x_list = []
        x_list.append(x_id)
        x_list.append(x_lab)
        x_list.append(x_time)
        listdata.append(x_list)
    names = ['SessionId', 'ItemId', 'TimeStamp']
    file = pd.DataFrame(columns = names, data = listdata)
    file.to_csv(data_path,index=False,sep='\t')
    print('transfor done')
    return 1

txt2csv(train_data, csv_train_path)
txt2csv(test_data, csv_test_path)

